# Experiment 1
This notebook contains the code for the first experiment of the paper. In this experiment, we train a B2BNet model that first encodes the temporal dimensions of the EEG data and then its spatial dimensions. It does not include any B2B or classification heads.

In [1]:
%reload_ext autoreload
%autoreload 3

import torch
import torch.nn.functional as F
from torchinfo import summary
import pytorch_lightning as pl
from src.b2bnet import B2BNetSpaceTimeModel, OtkaDataModule, OtkaTimeDimSplit
from pytorch_lightning.loggers import TensorBoardLogger

In [2]:
# Parameters
segment_size = 120 * 3 # 3sec
batch_size = 256
n_channels = 59
space_embedding_dim = 8
time_embedding_dim = 16
max_epochs = 1000

In [5]:
datamodule = OtkaTimeDimSplit(segment_size=segment_size, batch_size=batch_size, b2b_data='hypnotist',
                              data_mode='crop')

model = B2BNetSpaceTimeModel(
    n_channels=n_channels, space_embedding_dim=space_embedding_dim,
    time_embedding_dim=time_embedding_dim, kernel_size=1, n_subjects=52, base='encoder', b2b='embedding')

trainer = pl.Trainer(max_epochs=max_epochs, accelerator='cpu', log_every_n_steps=1, deterministic='warn')

trainer.fit(model, datamodule=datamodule)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


>>>>>> Using hypnotist data in b2b head
X_b2b shape: torch.Size([51, 39707, 59]) segment_size: 360



  | Name     | Type             | Params
----------------------------------------------
0 | base     | SpaceTimeEncoder | 2.8 K 
1 | b2b_head | SpaceTimeEncoder | 2.8 K 
2 | cls      | Linear           | 34    
----------------------------------------------
5.6 K     Trainable params
0         Non-trainable params
5.6 K     Total params
0.022     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
